# Setting Up the Environment

- Install FastAPI and Uvicorn<br>
FastAPI itself is only the framework. Uvicorn is an ASGI server to serve FastAPI apps.

In [ ]:
pip install fastapi uvicorn

# Build First FastAPI Application

### Create a File main.py

In [ ]:
from fastapi import FastAPI

# Create a FastAPI instance
app = FastAPI()

# Define a root endpoint
@app.get("/")
def read_root():
    return {"message": "Welcome to FastAPI!"}

# Define an endpoint with a path parameter and optional query parameter
@app.get("/items/{item_id}")
def read_item(item_id: int, q: str = None):
    return {"item_id": item_id, "query": q}

### Run the Server

In [ ]:
uvicorn main:app --reload

### Explanation:
- **uvicorn main:app** tells Uvicorn to run the app instance from the main.py file.

- **--reload** enables automatic reloading when you make code changes.

- Open your browser and navigate to:

Swagger UI: http://127.0.0.1:8000/docs

ReDoc: http://127.0.0.1:8000/redoc

# HTTP Method Decorators

### @app.get()
- Handles HTTP GET requests.
- Used to retrieve data.

In [ ]:
@app.get("/items/")
def read_items():
    return {"message": "GET request"}

### @app.post()
- Handles HTTP POST requests.
- Used to create new resources or submit data.

In [ ]:
@app.post("/items/")
def create_item(item: dict):
    return {"message": "POST request", "item": item}

### @app.put()
- Handles HTTP PUT requests.
- Used to update existing resources.

In [ ]:
@app.put("/items/{item_id}")
def update_item(item_id: int, item: dict):
    return {"message": f"PUT request for item {item_id}", "item": item}

### @app.patch()
- Handles HTTP PATCH requests.
- Used for partial updates of resources.

In [ ]:
@app.patch("/items/{item_id}")
def partial_update_item(item_id: int, item: dict):
    return {"message": f"PATCH request for item {item_id}", "updates": item}

### @app.delete()
- Handles HTTP DELETE requests.
- Used to delete a resource.

In [ ]:
@app.delete("/items/{item_id}")
def delete_item(item_id: int):
    return {"message": f"DELETE request for item {item_id}"}

### @app.options()
- Handles HTTP OPTIONS requests.
- Used to describe available methods and operations for a resource.

In [ ]:
@app.options("/items/")
def options_items():
    return {"message": "OPTIONS request"}

### @app.head()
- Handles HTTP HEAD requests.
- Similar to GET but returns only headers, no body.

In [ ]:
@app.head("/items/")
def head_items():
    return {"message": "HEAD request"}

### @app.trace()
- Handles HTTP TRACE requests.
- Used for diagnostic purposes to trace how a request is processed.

In [ ]:
@app.trace("/items/")
def trace_items():
    return {"message": "TRACE request"}

## Additional Routing Features

### Combine Multiple HTTP Methods
Use methods argument to handle multiple methods in one route.

In [ ]:
@app.api_route("/items/", methods=["GET", "POST"])
def handle_items():
    return {"message": "GET or POST request"}

### Include Metadata
Add summary and description to document the route in OpenAPI.

In [ ]:
@app.get("/items/", summary="Retrieve items", description="Fetches all items from the database.")
def read_items():
    return {"message": "GET request"}

### Tags
Group routes together with tags for better documentation.

In [ ]:
@app.get("/users/", tags=["Users"])
def get_users():
    return {"message": "List of users"}

## @app.on_event Decorators

The **@app.on_event** decorator is used to execute specific actions during **startup** or **shutdown** events of the FastAPI application. This is useful for tasks like connecting to a database, setting up caches, or closing resources.

Is intended to define global startup logic for the entire application, not individual routes.<br>
If you try to define multiple startup functions, they will all run when the application starts.

### Startup Event
The startup event is triggered when the FastAPI app starts running. This is often used for:

- Connecting to a database
- Initializing background tasks
- Preloading data or resources into memory

#### Example:

In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.on_event("startup")
async def startup_event():
    print("Application is starting up...")
    # Example: Connect to database
    # await database.connect()

### Shutdown Event
The shutdown event is triggered when the FastAPI app stops running. This is often used for:

- Closing database connections
- Cleaning up resources
- Saving any final logs or states

#### Example:

In [ ]:
@app.on_event("shutdown")
async def shutdown_event():
    print("Application is shutting down...")
    # Example: Disconnect from database
    # await database.disconnect()

# Path Parameters and Query Parameters

FastAPI supports path and query parameters seamlessly.
### Example:

In [ ]:
# Path parameter example
@app.get("/users/{user_id}")
def get_user(user_id: int):
    return {"user_id": user_id}

# Query parameter example
@app.get("/search")
def search_items(keyword: str, limit: int = 10):
    return {"keyword": keyword, "limit": limit}

### Explanation:
- **user_id** is a path parameter that must be included in the URL.
- **keyword** and **limit** are query parameters, and limit has a default value of 10.

<table border="1">
  <thead>
    <tr>
      <th>Aspect</th>
      <th>Path Parameter</th>
      <th>Query Parameter</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><strong>Location</strong></td>
      <td>Part of the URL path (e.g., /users/42)</td>
      <td>Part of the query string (e.g., ?key=value)</td>
    </tr>
    <tr>
      <td><strong>Usage</strong></td>
      <td>Identifies a specific resource (e.g., user_id)</td>
      <td>Used for filtering, searching, or passing optional data (e.g., limit, keyword)</td>
    </tr>
    <tr>
      <td><strong>Mandatory/Optional</strong></td>
      <td>Always mandatory</td>
      <td>Can be optional or have default values</td>
    </tr>
    <tr>
      <td><strong>Parsing</strong></td>
      <td>Extracted from the URL path</td>
      <td>Extracted from the query string after `?`</td>
    </tr>
    <tr>
      <td><strong>Example</strong></td>
      <td>/users/{user_id}</td>
      <td>/search?keyword=fastapi&limit=10</td>
    </tr>
    <tr>
      <td><strong>Purpose</strong></td>
      <td>To identify a specific entity or resource</td>
      <td>To customize, filter, or control the behavior of the request</td>
    </tr>
  </tbody>
</table>

# Request and Response Models

Use Pydantic models to define request and response schemas.

### Example:

In [ ]:
from pydantic import BaseModel

# Define a Pydantic model for the request body
class Item(BaseModel):
    name: str
    description: str = None
    price: float
    tax: float = None

# Use the model in an endpoint
@app.post("/items/")
def create_item(item: Item):
    total_price = item.price + (item.tax or 0)
    return {"item": item, "total_price": total_price}

### Explanation:
- The Item model defines the expected structure of the request body.
- FastAPI automatically validates the request and generates API documentation.

# Handling Validation

FastAPI validates data automatically using Pydantic models and type hints.

### Example:

In [ ]:
@app.get("/validate/{age}")
def validate_age(age: int):
    if age < 18:
        return {"error": "Underage"}
    return {"message": "Allowed"}

### Explanation:
FastAPI ensures age is an integer. If invalid, it returns a 422 error automatically.

# Authentication and Security

- FastAPI provides built-in tools for authentication and security.
- This flow allows you to authenticate users by sending a username and password and then receiving a token in return, which is then used for further authentication on protected routes.

### OAuth2PasswordBearer:
This is a class provided by FastAPI that expects an **OAuth2 token** to be passed in a request (typically via the Authorization header in the form of a **Bearer token**).
You use **OAuth2PasswordBeare**r to define a standard for how the client sends the token (usually through a **header**).

### tokenUrl:
The tokenUrl parameter in OAuth2PasswordBearer specifies the endpoint that the client will use to obtain the token. For instance, tokenUrl="token" means that there should be a route (/token) where the client can send their credentials (like a username and password) to get the token.

### The Depends Function:
FastAPI's Depends function is used to declare that a certain dependency is required. In this case, **token: str = Depends(oauth2_scheme)** tells FastAPI that the endpoint **read_users_me** requires the **token parameter**, which will be automatically populated with the OAuth2 token extracted from the Authorization header.
FastAPI will call the **oauth2_scheme** dependency, which is an instance of **OAuth2PasswordBearer**, to extract the token from the request.

### Authorization in Header:
The client will send the token in the Authorization header as a Bearer token:<pre>

Authorization: Bearer <your-token>
</pre>
FastAPI will extract the token from the header and pass it to the token argument in the read_users_me function.

### Example in Action
Let's break it down with a practical example:

#### Step 1: Token Generation
You first need a route that will authenticate a user and issue a token. A typical implementation might look like this:

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.security import OAuth2PasswordBearer
from fastapi import Depends, HTTPException

app = FastAPI()

# OAuth2 password flow
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# Simulating a database of users (for demonstration purposes)
fake_users_db = {
    "johndoe": {
        "username": "johndoe",
        "password": "secret",  # In a real application, passwords should be hashed
    }
}

# Request body for receiving username and password
class TokenRequest(BaseModel):
    username: str
    password: str

# Token generation endpoint (this is where the user gets a token)
@app.post("/token")
def login_for_access_token(form_data: TokenRequest):
    user = fake_users_db.get(form_data.username)
    if user is None or user["password"] != form_data.password:
        raise HTTPException(status_code=401, detail="Invalid credentials")
    # For simplicity, returning the username as the token
    return {"access_token": form_data.username, "token_type": "bearer"}




How It Works Internally:
The client sends the username and password to the /token endpoint.
The server verifies the credentials and returns an access token (in this case, a simple string for demonstration).
The client then sends this token in the Authorization header in subsequent requests.
FastAPI extracts the token from the header using the OAuth2PasswordBearer dependency.
FastAPI can then use this token to validate the user and access protected endpoints.

#### Step 2: Accessing Protected Route
Now, you can use the oauth2_scheme to require authentication on protected routes:

In [ ]:
@app.get("/users/me")
def read_users_me(token: str = Depends(oauth2_scheme)):
    # In a real app, you would validate the token and decode it
    return {"token": token}

##### Here:
- The read_users_me endpoint requires a valid token to access it.
- FastAPI automatically pulls the token from the Authorization header using the oauth2_scheme dependency and provides it to the token parameter in read_users_me.

#### Step 3: Making Requests


**Get the Token: To get a token, make a POST request to /token with your username and password in the body:**

- Request:<pre>

POST /token HTTP/1.1<br>
Content-Type: application/json<br>

{
    "username": "johndoe",<br>
    "password": "secret"<br>
}
</pre>
- Response:<pre>

{
    "access_token": "johndoe",<br>
    "token_type": "bearer"<br>
}
</pre>

**Access Protected Route: To access /users/me, include the Bearer token in the Authorization header:**

- Request:<pre>

GET /users/me HTTP/1.1<br>
Authorization: Bearer johndoe<br>
</pre>
- Response:<pre>

{
    "token": "johndoe"
}

###  How It Works Internally:
- The client sends the username and password to the /token endpoint.
- The server verifies the credentials and returns an access token (in this case, a simple string for demonstration).
- The client then sends this token in the Authorization header in subsequent requests.
- FastAPI extracts the token from the header using the OAuth2PasswordBearer dependency.
- FastAPI can then use this token to validate the user and access protected endpoints.

# Dependency Injection

FastAPI supports dependency injection to manage shared functionality.

### Example:

In [ ]:
from fastapi import Depends
import sqlite3

# Dependency that simulates a database connection
def get_db():
    db = sqlite3.connect('mydatabase.db')
    try:
        yield db  # Yield the database connection to be used by the route
    finally:
        db.close()  # Close the connection when the request is finished

# Route that uses the database connection
@app.get("/users/{user_id}")
def read_user(user_id: int, db: sqlite3.Connection = Depends(get_db)):
    cursor = db.cursor()
    cursor.execute("SELECT * FROM users WHERE id=?", (user_id,))
    user = cursor.fetchone()
    return {"user_id": user_id, "user": user}

### Explanation:
- Depends allows you to inject shared logic into your endpoints.

# Middleware and Background Tasks

## Middleware
is a mechanism for processing requests before they reach the endpoint and responses before they are sent back to the client

### Example:
FastAPI provides a way to add middleware using the **BaseHTTPMiddleware** class. You can also use third-party middleware.

In [ ]:
from fastapi import FastAPI
from starlette.middleware.base import BaseHTTPMiddleware

app = FastAPI()

class SimpleMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request, call_next):
        # Modify request here, before it goes to the route handler
        print("Request received")
        
        # Process the request and get the response
        response = await call_next(request)
        
        # Modify the response before sending it back
        response.headers["X-Custom-Header"] = "Middleware"
        
        return response

# Add the middleware to the app
app.add_middleware(SimpleMiddleware)

@app.get("/")
async def read_root():
    return {"message": "Hello, FastAPI with middleware!"}

### Explanation:

- **Middleware** allows you to modify requests and responses globally.
- The **CORS** middleware enables cross-origin requests.

## Background Task
is used for performing tasks asynchronously after a response has been sent to the client. such as sending emails, processing files, or making external API calls.

### Example:
FastAPI provides a built-in **BackgroundTasks** class to manage these tasks.

In [ ]:
from fastapi import FastAPI, BackgroundTasks
import time

app = FastAPI()

# Background task function
def write_log(message: str):
    time.sleep(5)  # Simulate a time-consuming task
    with open("log.txt", mode="a") as log:
        log.write(message + "\n")

@app.get("/")
async def root(background_tasks: BackgroundTasks):
    background_tasks.add_task(write_log, "Task completed!")
    return {"message": "The background task is being executed."}

### Explanation:
- **BackgroundTasks**: This class is injected as a parameter into the endpoint function. You can then add tasks to be executed in the background using background_tasks.add_task().
- **write_log**: A simple background task that simulates writing a log message after a delay. The task is executed in the background without blocking the response.
- **background_tasks.add_task(write_log, "Task completed!")**: The task write_log is added to the background and will be executed asynchronously once the response is sent back to the client.

# File Upload and Streaming

via **Form** and **File** dependencies. You can upload images, videos, or any other file.

In [ ]:
from fastapi import File, UploadFile

@app.post("/uploadfile/")
async def upload_file(file: UploadFile = File(...)):
    return {"filename": file.filename}

# Testing Your FastAPI Application

FastAPI provides a great way to test your application using **TestClient**.

### Install Required Dependencies

In [ ]:
pip install httpx pytest

### Example:

In [ ]:
from fastapi import FastAPI
from fastapi.testclient import TestClient

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Hello, FastAPI!"}

# Testing with TestClient
def test_read_root():
    client = TestClient(app)
    response = client.get("/")
    assert response.status_code == 200
    assert response.json() == {"message": "Hello, FastAPI!"}

### Testing with Query Parameters

In [ ]:
@app.get("/search")
def search_items(keyword: str, limit: int = 10):
    return {"keyword": keyword, "limit": limit}

def test_search_items():
    client = TestClient(app)
    response = client.get("/search?keyword=test&limit=5")
    assert response.status_code == 200
    assert response.json() == {"keyword": "test", "limit": 5}

### Testing with Path Parameters

In [ ]:
@app.get("/items/{item_id}")
def read_item(item_id: int):
    return {"item_id": item_id}

def test_read_item():
    client = TestClient(app)
    response = client.get("/items/123")
    assert response.status_code == 200
    assert response.json() == {"item_id": 123}

### Testing with Request Body (POST request)

In [ ]:
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    price: float

@app.post("/items/")
def create_item(item: Item):
    return {"name": item.name, "price": item.price}

def test_create_item():
    client = TestClient(app)
    item_data = {"name": "Laptop", "price": 1000}
    response = client.post("/items/", json=item_data)
    assert response.status_code == 200
    assert response.json() == item_data

### Testing Authentication and Dependencies

In [ ]:
from fastapi import Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

def fake_oauth2_scheme():
    return "fake_token"

@app.get("/users/me")
def read_users_me(token: str = Depends(fake_oauth2_scheme)):
    if token != "fake_token":
        raise HTTPException(status_code=status.HTTP_401_UNAUTHORIZED, detail="Invalid token")
    return {"token": token}

def test_read_users_me():
    client = TestClient(app)
    response = client.get("/users/me", headers={"Authorization": "Bearer fake_token"})
    assert response.status_code == 200
    assert response.json() == {"token": "fake_token"}

### Running Tests with pytest
You can run the tests using pytest, which will automatically find and run all functions prefixed with test_:

In [ ]:
pytest

# Deploying FastAPI

Using Uvicorn with Gunicorn

Install Gunicorn:

In [ ]:
pip install gunicorn

Run the server:

In [ ]:
gunicorn -k uvicorn